In [1]:
# Import Dependencies and set variables
import pandas as pd
from pycoingecko import CoinGeckoAPI
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
# Set Variables
currency = 'usd'
from_date = datetime(2015, 1, 1)
end_date = datetime(2021, 9, 30)
cg = CoinGeckoAPI()

In [3]:
# Functions
def convert_timestamp(df):
    df['date'] = df['date'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
    df['date'] = pd.to_datetime(df.date)
    

## Pull Exchanges from API

In [49]:
# Gather list of exchanges
exchanges_list = cg.get_exchanges_list()
exchanges_list[0]

{'id': 'binance',
 'name': 'Binance',
 'year_established': 2017,
 'country': 'Cayman Islands',
 'description': '',
 'url': 'https://www.binance.com/',
 'image': 'https://assets.coingecko.com/markets/images/52/small/binance.jpg?1519353250',
 'has_trading_incentive': False,
 'trust_score': 10,
 'trust_score_rank': 1,
 'trade_volume_24h_btc': 540673.0067455786,
 'trade_volume_24h_btc_normalized': 540673.0067455786}

In [50]:
# Convert exchanges list into dataframe
exchanges_df = pd.DataFrame(exchanges_list).drop(columns=['description', 'url', 'image', 'has_trading_incentive',
    'trust_score', 'trust_score_rank', 'trade_volume_24h_btc_normalized']).set_index('id').\
    sort_values(by=['trade_volume_24h_btc'], ascending=False)

exchanges_df

,name,year_established,country,trade_volume_24h_btc
id,,,,
binance,Binance,2017.0,Cayman Islands,540673.006746
okex,OKEx,2013.0,Belize,141065.526073
gdax,Coinbase Exchange,2012.0,United States,135543.090218
coinflex,CoinFLEX,2019.0,Seychelles,125224.640306
zbg,ZBG,2018.0,China,110585.703678
...,...,...,...,...
narkasa,Narkasa,2020.0,Turkey,49.096765
nice_hash,NiceHash,NaN,Slovenia,46.316635
kuna,Kuna Exchange,NaN,United Kingdom,29.073133


## Analyze Exchanges DataFrame

In [51]:
# Find dtypes of Exchanges DataFrame and change if necessary
print('dtypes before >>>\n')
print(exchanges_df.dtypes)

exchanges_df = exchanges_df.convert_dtypes()

print('\ndtypes after >>>\n')
print(exchanges_df.dtypes)

dtypes before >>>

name                     object
year_established        float64
country                  object
trade_volume_24h_btc    float64
dtype: object

dtypes after >>>

name                     string
year_established          Int64
country                  string
trade_volume_24h_btc    Float64
dtype: object


In [52]:
# Find and Fill NA Values
print(exchanges_df.info(verbose = True))

# Fill/Drop NA columns
exchanges_df.country = exchanges_df.country.fillna('No Country Provided')
exchanges_df = exchanges_df.dropna()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, binance to biki
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  100 non-null    string 
 1   year_established      91 non-null     Int64  
 2   country               89 non-null     string 
 3   trade_volume_24h_btc  100 non-null    Float64
dtypes: Float64(1), Int64(1), string(2)
memory usage: 4.1+ KB
None


In [53]:
# Describe columns that are numeric
print(exchanges_df.describe())

       year_established  trade_volume_24h_btc
count         91.000000             91.000000
mean        2016.703297          24122.050450
std            2.451732          62114.164394
min         2011.000000              3.273088
25%         2014.000000           1793.929748
50%         2018.000000           5513.141002
75%         2018.000000          21204.337414
max         2020.000000         540673.006746


In [54]:
# Only keep exchanges in the top 50 in terms of 24h btc trade volume
top50=exchanges_df['trade_volume_24h_btc'].quantile(.50)
print(top50)
exchanges_df = exchanges_df[exchanges_df['trade_volume_24h_btc'] >= top50]

5513.141002018656


In [55]:
# Correctly Name Colums and Final Formatting
exchanges_df = exchanges_df.rename(columns={'name': 'Name', 'year_established': 'Year_Established', 'country': 'Country', 'trade_volume_24h_btc': 'BTC_24hVolume'})
exchanges_df.index = exchanges_df.index.rename('ExchangeID')

exchanges_df

,Name,Year_Established,Country,BTC_24hVolume
ExchangeID,,,,
binance,Binance,2017,Cayman Islands,540673.006746
okex,OKEx,2013,Belize,141065.526073
gdax,Coinbase Exchange,2012,United States,135543.090218
coinflex,CoinFLEX,2019,Seychelles,125224.640306
zbg,ZBG,2018,China,110585.703678
huobi,Huobi Global,2013,Seychelles,89460.715968
upbit,Upbit,2017,South Korea,62734.680732
kucoin,KuCoin,2014,Seychelles,59719.6856
ftx_spot,FTX,2019,Antigua and Barbuda,52419.278481


In [56]:
# Gather list of Exchange IDs for later
exchange_id_list = list(exchanges_df.index)
print(len(exchange_id_list))
exchange_id_list

46


['binance',
 'okex',
 'gdax',
 'coinflex',
 'zbg',
 'huobi',
 'upbit',
 'kucoin',
 'ftx_spot',
 'hitbtc',
 'bitrue',
 'bitcoin_com',
 'changelly',
 'crypto_com',
 'bkex',
 'uniswap',
 'xt',
 'zb',
 'kraken',
 'digifinex',
 'bitmart',
 'binance_us',
 'bithumb',
 'p2pb2b',
 'bigone',
 'bitfinex',
 'pancakeswap_new',
 'mxc',
 'whitebit',
 'lbank',
 'btcturk',
 'bitforex',
 'wootrade',
 'coinbene',
 'sushiswap',
 'uniswap_v2',
 'latoken',
 'aax',
 'phemex',
 'goku',
 'catex',
 'bitstamp',
 'ftx_us',
 'bitexen',
 'bithumb_global',
 'cointiger']

## Pull ticker data from API

In [57]:
# Gather ticker data for exchanges in list
n = 0
for exchange in exchange_id_list:
    print(f'Gathering Data for {exchange}...')
    
    exchange_data = cg.get_exchanges_tickers_by_id(id = exchange)
    ticker_data = exchange_data['tickers']
    temp_df = pd.DataFrame(ticker_data)
    
    # Drop Columns
    columns_to_drop = ['market', 'bid_ask_spread_percentage',  'converted_last',
           'converted_volume', 'trust_score',
           'last_traded_at', 'last_fetch_at', 'is_anomaly',
           'is_stale', 'trade_url', 'token_info_url']
    temp_df = temp_df.drop(columns = columns_to_drop)
    
    # Format Columns
    temp_df['ExchangeID'] = exchange
    temp_df = temp_df.rename(columns = {'base': 'Ticker', 'last': 'LastPrice', 'volume': 'Volume', 
                                              'timestamp': 'Time', 'coin_id':'CoinID'})
    temp_df.index.rename('TickerID', inplace=True)
    
    if n == 0:
        tickers_df = temp_df
        n += 1
    else:
        tickers_df = tickers_df.append(temp_df, ignore_index=True)
        
print('Finished Gathering Data!')

Gathering Data for binance...
Gathering Data for okex...
Gathering Data for gdax...
Gathering Data for coinflex...
Gathering Data for zbg...
Gathering Data for huobi...
Gathering Data for upbit...
Gathering Data for kucoin...
Gathering Data for ftx_spot...
Gathering Data for hitbtc...
Gathering Data for bitrue...
Gathering Data for bitcoin_com...
Gathering Data for changelly...
Gathering Data for crypto_com...
Gathering Data for bkex...
Gathering Data for uniswap...
Gathering Data for xt...
Gathering Data for zb...
Gathering Data for kraken...
Gathering Data for digifinex...
Gathering Data for bitmart...
Gathering Data for binance_us...
Gathering Data for bithumb...
Gathering Data for p2pb2b...
Gathering Data for bigone...
Gathering Data for bitfinex...
Gathering Data for pancakeswap_new...
Gathering Data for mxc...
Gathering Data for whitebit...
Gathering Data for lbank...
Gathering Data for btcturk...
Gathering Data for bitforex...
Gathering Data for wootrade...
Gathering Data for co

In [58]:
# View Tickers Table
tickers_df

,Ticker,target,LastPrice,Volume,Time,CoinID,target_coin_id,ExchangeID
0,BUSD,USDT,0.999900,1.005246e+09,2021-10-27T06:46:18+00:00,binance-usd,tether,binance
1,BTC,USDT,60750.350000,4.710408e+04,2021-10-27T06:49:24+00:00,bitcoin,tether,binance
2,ETH,USDT,4245.510000,5.273222e+05,2021-10-27T06:54:04+00:00,ethereum,tether,binance
3,SHIB,USDT,0.000054,6.966767e+13,2021-10-27T06:43:58+00:00,shiba-inu,tether,binance
4,BTC,BUSD,60739.070000,1.243522e+04,2021-10-27T06:57:01+00:00,bitcoin,binance-usd,binance
...,...,...,...,...,...,...,...,...
4242,ICX,BTC,0.000033,3.670265e+02,2021-10-27T06:53:55+00:00,icon,bitcoin,cointiger
4243,REP,BTC,0.000390,6.954816e+01,2021-10-27T07:05:58+00:00,augur,bitcoin,cointiger
4244,BTM,ETH,0.000012,1.889471e+04,2021-10-27T05:25:23+00:00,bytom,ethereum,cointiger
4245,GZX,USDT,0.000251,2.451909e+09,2021-10-27T06:40:23+00:00,greenzonex,tether,cointiger


## Analyze Tickers DataFrame

In [59]:
# Find Target Value Counts to keep only USD and USD stablecoins to have value in USD
target_value_counts = tickers_df.target.value_counts()
print(f'The Value Counts for Target Tickers are \n{target_value_counts}')

The Value Counts for Target Tickers are 
USDT                                          2017
BTC                                            618
USD                                            368
ETH                                            231
KRW                                            168
                                              ... 
0XDBDB4D16EDA451D0503B854CF79D55697F90C8DF       1
0XFCA59CD816AB1EAD66534D82BC21E7515CE441CF       1
0XCFEB09C3C5F0F78AD72166D55F9E6E9A60E96EEC       1
0XCD7492DB29E2AB436E819B249452EE1BBDF52214       1
0XF5581DFEFD8FB0E4AEC526BE659CFAB1F8C781DA       1
Name: target, Length: 110, dtype: int64


In [60]:
# Remove all non USD and USD stablecoins from dataframe
tickers_df = tickers_df[(tickers_df['target'] == 'USDT') | (tickers_df['target'] == 'USD') | (tickers_df['target'] == 'USDC') | (tickers_df['target'] == 'BUSD') | (tickers_df['target'] == 'TUSD') | (tickers_df['target'] == 'HUSD')]

tickers_df.target.value_counts()

USDT    2017
USD      368
USDC      81
BUSD      62
TUSD      11
HUSD       3
Name: target, dtype: int64

In [61]:
# Remove Target and Target Coin ID as it is unnecessary for analysis and rename index
tickers_df = tickers_df.drop(columns=['target', 'target_coin_id'])
tickers_df.index.rename('TickerID', inplace=True)

In [65]:
print(tickers_df.Ticker.value_counts())

BTC                                           75
ETH                                           73
LTC                                           50
BCH                                           42
LINK                                          42
                                              ..
STSOL                                          1
GTC                                            1
RAZOR                                          1
RACA                                           1
0X853D955ACEF822DB058EB8505911ED77F175B99E     1
Name: Ticker, Length: 476, dtype: int64


## Pull API Data for Exchange Volumes

In [45]:
cg.get_exchanges_volume_chart_by_id(id='kraken', days=365)


[[1603867800000.0,
  '26092.626279693087682052893563567197577064486278257554494'],
 [1603954200000.0,
  '26295.244319990599158415653159248657802324576003171064545'],
 [1604040600000.0,
  '22480.319018675747195429885776618848286601002174356379566'],
 [1604127000000.0, '19053.29703769689569528957863387598846474838792843070513'],
 [1604213400000.0,
  '15942.972267449587273476806589451773912838601206278075784'],
 [1604299800000.0, '9493.98611988797925940612910772941212031516559429967522'],
 [1604386200000.0,
  '21961.006922208620142954559525821303737782063151984604182'],
 [1604472600000.0,
  '19511.375568556247469525548438860733435659533058084148748'],
 [1604559000000.0,
  '23908.120713848070668391653038003747123486728330499118335'],
 [1604645400000.0, '46769.92022802921562885608367776213671617880404781392632'],
 [1604731800000.0, '30928.48819800135292599432787875257284973278405330492545'],
 [1604818200000.0, '30691.26485744592313328943210610181824882360072125265407'],
 [1604904600000.0,
 

## Gather List of Coins

In [17]:
# Gather coin list
coins_list = cg.get_coins_list()
coins_list

[{'id': '01coin', 'symbol': 'zoc', 'name': '01coin'},
 {'id': '0-5x-long-algorand-token',
  'symbol': 'algohalf',
  'name': '0.5X Long Algorand Token'},
 {'id': '0-5x-long-altcoin-index-token',
  'symbol': 'althalf',
  'name': '0.5X Long Altcoin Index Token'},
 {'id': '0-5x-long-balancer-token',
  'symbol': 'balhalf',
  'name': '0.5X Long Balancer Token'},
 {'id': '0-5x-long-bitcoin-cash-token',
  'symbol': 'bchhalf',
  'name': '0.5X Long Bitcoin Cash Token'},
 {'id': '0-5x-long-bitcoin-sv-token',
  'symbol': 'bsvhalf',
  'name': '0.5X Long Bitcoin SV Token'},
 {'id': '0-5x-long-bitcoin-token',
  'symbol': 'half',
  'name': '0.5X Long Bitcoin Token'},
 {'id': '0-5x-long-cardano-token',
  'symbol': 'adahalf',
  'name': '0.5X Long Cardano Token'},
 {'id': '0-5x-long-chainlink-token',
  'symbol': 'linkhalf',
  'name': '0.5X Long Chainlink Token'},
 {'id': '0-5x-long-cosmos-token',
  'symbol': 'atomhalf',
  'name': '0.5X Long Cosmos Token'},
 {'id': '0-5x-long-defi-index-token',
  'symbol'

In [18]:
# Convert Coins list into dataframe
coins_df = pd.DataFrame(coins_list).set_index('id')
coins_df

,symbol,name
id,,
01coin,zoc,01coin
0-5x-long-algorand-token,algohalf,0.5X Long Algorand Token
0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index Token
0-5x-long-balancer-token,balhalf,0.5X Long Balancer Token
0-5x-long-bitcoin-cash-token,bchhalf,0.5X Long Bitcoin Cash Token
...,...,...
zyro,zyro,Zyro
zytara-dollar,zusd,Zytara Dollar
zyx,zyx,ZYX


In [19]:
# Get OHLC data
ohlc_data = cg.get_coin_ohlc_by_id(id = 'bitcoin', vs_currency = currency, days = 'max')
ohlc_data

[[1367280000000, 135.3, 141.96, 135.3, 135.3],
 [1367539200000, 117.0, 117.0, 91.01, 91.01],
 [1367884800000, 111.25, 118.33, 106.4, 106.4],
 [1368230400000, 112.64, 118.78, 112.64, 113.01],
 [1368576000000, 114.71, 117.18, 114.16, 114.16],
 [1368921600000, 115.5, 123.88, 115.5, 120.5],
 [1369267200000, 122.58, 125.75, 122.58, 125.75],
 [1369612800000, 131.7, 134.6, 128.99, 128.99],
 [1369958400000, 129.18, 132.13, 127.4, 127.98],
 [1370217600000, 129.09, 129.09, 121.16, 121.16],
 [1370563200000, 121.31, 121.71, 119.0, 119.0],
 [1370908800000, 110.09, 110.09, 99.75, 105.99],
 [1371254400000, 108.9, 108.9, 98.4, 98.4],
 [1371600000000, 99.99, 106.55, 99.99, 106.55],
 [1371945600000, 107.9, 110.91, 107.25, 107.25],
 [1372291200000, 108.0, 108.0, 102.44, 103.39],
 [1372550400000, 99.66, 99.66, 93.33, 95.39],
 [1372809600000, 97.5, 97.5, 89.49, 90.8],
 [1373155200000, 78.8, 78.8, 67.81, 68.08],
 [1373500800000, 77.5, 85.75, 76.39, 85.75],
 [1373846400000, 93.8, 95.58, 88.06, 94.0],
 [13741

In [20]:
# Format OHLC Data as DataFrame
ohlc_df = pd.DataFrame(ohlc_data, columns = ['date', 'open', 'high', 'low', 'close'])

# Fix and Filter Dates
ohlc_df['date'] = ohlc_df['date'].apply(lambda x: int(str(x)[:-3]))
convert_timestamp(ohlc_df)
ohlc_df = ohlc_df.loc[(ohlc_df['date'] >= from_date) & (ohlc_df['date'] <= end_date)]

# Add Coin ID Columns and Format Columns
ohlc_df['coin_id'] = 'bitcoin'
ohlc_df = ohlc_df.set_index('date')
ohlc_df.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'coin_id': 'CoinID'}, inplace=True)
ohlc_df.index.rename('Date', inplace=True)
ohlc_df

,Open,High,Low,Close,CoinID
Date,,,,,
2015-01-02,313.99,314.45,286.57,286.57,bitcoin
2015-01-06,260.94,295.87,260.94,295.87,bitcoin
2015-01-10,284.45,289.57,267.18,267.18,bitcoin
2015-01-14,268.35,268.35,172.15,210.56,bitcoin
2015-01-18,205.82,214.55,199.49,214.55,bitcoin
...,...,...,...,...,...
2021-09-14,45196.49,47135.43,45139.83,47135.43,bitcoin
2021-09-18,48186.53,48266.63,47228.66,48266.63,bitcoin
2021-09-22,47371.04,47371.04,40386.62,43587.22,bitcoin


In [21]:
# Get Historical Market Data
raw_market_data = cg.get_coin_market_chart_by_id(id = 'bitcoin', vs_currency = currency, days = 'max')
raw_market_data

{'prices': [[1367107200000, 135.3],
  [1367193600000, 141.96],
  [1367280000000, 135.3],
  [1367366400000, 117.0],
  [1367452800000, 103.43],
  [1367539200000, 91.01],
  [1367625600000, 111.25],
  [1367712000000, 116.79],
  [1367798400000, 118.33],
  [1367884800000, 106.4],
  [1367971200000, 112.64],
  [1368057600000, 113.0],
  [1368144000000, 118.78],
  [1368230400000, 113.01],
  [1368316800000, 114.713],
  [1368403200000, 117.18],
  [1368489600000, 114.5],
  [1368576000000, 114.156],
  [1368662400000, 115.5],
  [1368748800000, 123.1],
  [1368835200000, 123.88],
  [1368921600000, 120.501],
  [1369008000000, 122.58],
  [1369094400000, 122.9],
  [1369180800000, 123.0],
  [1369267200000, 125.748],
  [1369353600000, 131.7],
  [1369440000000, 130.77],
  [1369526400000, 134.6],
  [1369612800000, 128.985],
  [1369699200000, 129.179],
  [1369785600000, 132.13],
  [1369872000000, 127.401],
  [1369958400000, 127.98],
  [1370044800000, 129.09],
  [1370131200000, 121.3],
  [1370217600000, 121.16]

In [22]:
market_data = raw_market_data
# Clean Up the Data
columns = ['prices', 'market_caps', 'total_volumes']
n = 0
timestamp_list = []
for column in columns:
    temp_list = market_data[column]
    column_list = []
    for item in temp_list:
        timestamp_list.append(str(item.pop(0)))
        column_list.append(item.pop())
    
    market_data[column] = column_list
    if n == 0:
        market_data['date'] = [x[:-3] for x in timestamp_list]
        n+=1

In [23]:
# Format market data into df
market_df = pd.DataFrame(market_data)
market_df['date'] = market_df['date'].apply(lambda d: datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))
market_df['date'] = pd.to_datetime(market_df.date)
market_df = market_df.loc[market_df['date'] >= from_date]
market_df = market_df.set_index('date')
market_df['coin_id'] = 'bitcoin'
market_df

,prices,market_caps,total_volumes,coin_id
date,,,,
2015-01-01,314.446000,4.301448e+09,3.885591e+07,bitcoin
2015-01-02,286.572000,3.921358e+09,1.187789e+08,bitcoin
2015-01-03,260.936000,3.571640e+09,2.055001e+08,bitcoin
2015-01-04,273.220000,3.740880e+09,1.550381e+08,bitcoin
2015-01-05,285.373800,3.908408e+09,9.700290e+07,bitcoin
...,...,...,...,...
2021-10-22,61029.270932,1.151509e+12,4.076646e+10,bitcoin
2021-10-23,61572.140434,1.161166e+12,2.850504e+10,bitcoin
2021-10-24,61173.166391,1.153315e+12,2.998254e+10,bitcoin
